## ВКР: сбор данных

Автоматизация подбора персонала

Копчев Владислав, БПМИ197

В этом ноутбуке мы осуществим сбор данных, которые в следующих ноутбуках будут предобработаны и проанализированы с помощью методов машинного обучения. 

По итогу мы получим две таблицы:
... see PP.

### Библиотеки

В данной работе мы будем работать с таблицами, векторами, регулярными выражениями, сайтами с использованием JavaScript. Поэтому импортируем `pandas`, `selenium`, `numpy`, etc.

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# !pip install transliterate
from transliterate import translit
import re
import numpy as np

### Скрейпинг страницы со списком резюме

hh.ru не принимает названия с заглавными буквами, поэтому была написана функция для "нормализации" запросов — приведения к единому виду, который понимает сайт (данный вид был найден эмпирически — перебором различных запросов):

In [2]:
def normalize_translit(ru_query):
    text = translit(ru_query, language_code='ru', reversed=True)
    text = text.replace(' ', '-').replace('yj', 'yy').replace('yy-', 'yy_').replace('es-', 'es_').replace('k-pr', 'k_pr')  # эвристическое правило, не знаю, почему у hh так...
    return text

Запустим хром для того, чтобы работать с ним через библиотеку `selenium`:

In [3]:
driver = webdriver.Chrome('./chromedriver')

/var/folders/0k/pk7zyw6x2sqcjpbqvgf0ssh80000gn/T/ipykernel_37833/2299280398.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


Напишем функцию для скрейпинга вакансий по запросу `query`. Функция принимает переменную `driver`, которую мы уже инициализировали выше, а также запрос `query`, который мы ищем на сайте с вакансиями, который необходимо предварительно нормализовать. Для начала мы ищем, сколько страниц с вакансиями было найдено по запросу. После этого мы перебираем все страницы и на каждой скрейпим ссылку и название, которые задаются с помощью `XPath` как:

- `//div[@class="resume-search-item__header"]`
- `//div[@data-qa="resume-serp__results-search"]`

В тексте ВКР по аналогии с курсовой будет приведена таблица со всеми такими запросами.

In [29]:
def scrape(query, driver):
    # ищем кол-во страниц
    pages_count = 10  # соскрейпить это число иначе ошибки!
    link = 'https://hh.ru/resumes/{}?items_on_page=100&page={}'.format(query, 0)
    driver.get(link)
    
    pages_cnt_site = driver.find_elements(by=By.XPATH, value='//span[@class="pager-item-not-in-short-range"]/a[@class="bloko-button"][@rel="nofollow"][@data-qa="pager-page"]/span')
    if pages_cnt_site:
        pages_count = int(pages_cnt_site[-1].text)
        # print(pages_count)
    
    for n in range(pages_count):  # проблема!!
        link = 'https://hh.ru/resumes/{}?items_on_page=100&page={}'.format(query, n)  # 100 per page
        driver.get(link)

        resumes_list = driver.find_element(by=By.XPATH, value='//div[@class="resume-search-item__header"]')
        # resumes_list.click()  # стоп а зачем это...
        resumes = resumes_list.find_element(by=By.XPATH, value='//div[@data-qa="resume-serp__results-search"]').find_elements(by=By.XPATH, value='//a[@class="serp-item__title"]')

        resumes_parsed = []
        for x in resumes:
            resumes_parsed.append((x.text, x.get_attribute("href")))

        if n == 0:
            df = pd.DataFrame(resumes_parsed, columns=['Вакансия', 'Ссылка'])
            # print(len(resumes_parsed))
        else:
            # print(len(resumes_parsed))
            df2 = pd.DataFrame(resumes_parsed, columns=['Вакансия', 'Ссылка'])  # , index=list(range(0 + 100 * n, 100 + 100 * n)) n00, ..., n99n = 0, ..., 49
            df = pd.concat([df, df2])
    
    return df

Мы написали функцию. Запустим ее на нескольких запросах: 

In [ ]:
analysts_list = ['аналитик bi',
                'системный аналитик',
                'бизнес аналитик',
                'аналитик продаж',
                'финансовый аналитик',
                'аналитик данных',
                'data analyst']
analysts_map = dict()

for analyst in analysts_list:
    analysts_map[analyst] = scrape(normalize_translit(analyst), driver)
    analysts_map[analyst].index = range(analysts_map[analyst].shape[0])

Мы получили для каждого запроса словарь вида `query` $\mapsto$ Названия и ссылки вакансий. Теперь мы каждую такую мапу переведем в датафрейм (таблицу) и объединим все таблицы. Конкатеннация результатов:

In [ ]:
df1 = pd.DataFrame()

for analyst in analysts_list:
    df2 = analysts_map[analyst]
    df1 = pd.concat([df1, df2])
    
df1.index = range(df1.shape[0])
df1.to_csv('resumes_all.csv')

Итог: собрано около 20,000 резюме по 7 запросам на разных языках.

### Скрейпинг страницы с конкретным резюме

Итак, у нас есть табличка с резюме. Теперь из каждого резюме, которое представляет собой неструктурированный текст, мы достанем информацию (решим задачу NER).

Для начала напишем функцию для обработки `selenium.object` — если ничего не найдено по `XPath`-запросу, то пишем пропуск в виде `---`, иначе берем текст первого найденного по запросу объекта на странице.

In [193]:
def normalize(x):
    if x:
        x = x[0].text
    else:
        x = '---'  # так будет лучше чем просто ' '?
    return x

Напишем функцию для скрейпинга страницы с конкретным резюме, на которое ведет ссылка `link`, которую мы открываем с помощью библиотеки `selenium` на основе хромдрайвера (написать точное определение из документации, что это) `driver`. Эта функция достает:
- Коммандировка
- Опыт
- ...
См. Project Proposal. Необходимо поудалять ненужные комментарии и составить таблицу (мб даже в Markdown здесь) с `XPath`-запросами для каждой информации о резюме.

In [202]:
def scrape(link, driver):
    driver.get(link)
    comandirovka = driver.find_elements(by=By.XPATH, value='//div[@class="bloko-translate-guard"]')
    comandirovka = normalize(comandirovka)  # норм?
    
    opyt = driver.find_elements(by=By.XPATH, value='//span[@class="resume-block__title-text resume-block__title-text_sub"]')
    opyt = normalize(opyt)

    about = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block-container"][@data-qa="resume-block-skills-content"]')
    about = normalize(about)
        
    educ = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block"][@data-qa="resume-block-education"]')
    educ = normalize(educ)
        
    inter = driver.find_elements(by=By.XPATH, value='//div[@class="resume-block-item-gap"]')
    inter = normalize(inter)
        
    #inter2 = driver.find_elements(by=By.XPATH, value='//div[@class="key-skills-row"][@data-qa="tags-key-skills"]')
    #inter2 = normalize(inter2)
    inter2 = driver.find_elements(by=By.XPATH, value='//div[@data-qa="skills-table"]')
    inter2 = normalize(inter2)
    
    # Образование; все написано не в том же стиле что и ^
    
    educ_val = '//div[@data-qa="resume-block-education"][@class="resume-block"]//div[@data-qa="resume-block-education-name"]'
    educ_ = driver.find_elements(by=By.XPATH, value=educ_val)
    stepen_val = '//div[@data-qa="resume-block-education"]//span[@class="resume-block__title-text resume-block__title-text_sub"]'
    stepen = driver.find_elements(by=By.XPATH, value=stepen_val)
    title_val = '//span[@class="resume-block__title-text"][@data-qa="resume-block-title-position"]'
    title = driver.find_elements(by=By.XPATH, value=title_val)
    if title:
        title = title[0].text
    else:
        title = ''
    
    educ1 = ''
    educ2 = ''
    educ3 = ''
    
    if educ_:
        educ1 = educ_[0].text
    if len(educ_) > 1:
        educ2 = educ_[1].text
    if stepen:
        educ3 = stepen[0].text
    vuzes_count = len(educ_)
    
    # Проблема: иногда уровень образования могут не писать
    
    # Работы
    work = driver.find_elements(by=By.XPATH, 
                             value='//div[@data-qa="resume-block-experience"][@class="resume-block"]//div[@class="resume-block-container"]')
    work1 = '---'
    work2 = '---'
    
    if len(work) > 0:
        work1 = work[0].text
    if len(work) > 1:
        work2 = work[1].text
    
    # EXPERIMENTAL: пробую собрать work
    work_list = []
    for w in work:
        work_list.append(w.text)
    work_count = len(work)
        
    return (link, title, comandirovka, opyt, about, educ, inter, inter2, educ1, educ2, 
            educ3, vuzes_count, work1, work2, work_list, work_count)

Теперь создадим datagrame на основе нашего списка:

In [204]:
df_links = list(pd.read_csv('resumes_all.csv')['Ссылка'])[:300]  # rm [:300]

resume_list = []
for link in df_links:
    # print(link)
    data = scrape(link, driver)
    resume_list.append(data)

df = pd.DataFrame(resume_list, columns=['Ссылка',
                                        'Название',
                                        'Коммандировка', 
                                        'Опыт работы, лет',
                                        'О себе',
                                        'Образование',
                                        'Интересы',
                                        'Навыки',
                                        'Образование-1',
                                        'Образование-2',
                                        'Уровень образование',
                                        'Кол-во образований',
                                        'Работа 1',
                                        'Работа 2',
                                        'Где работал?',
                                        'Кол-во работ'])
df.to_csv('resumes_features.csv')
df.head(2)

KeyboardInterrupt: 

Это просто случайно продублировалось??

In [205]:
df = pd.DataFrame(resume_list, columns=['Ссылка',
                                        'Название',
                                        'Коммандировка', 
                                        'Опыт работы, лет',
                                        'О себе',
                                        'Образование',
                                        'Интересы',
                                        'Навыки',
                                        'Образование-1',
                                        'Образование-2',
                                        'Уровень образование',
                                        'Кол-во образований',
                                        'Работа 1',
                                        'Работа 2',
                                        'Где работал?',
                                        'Кол-во работ'])
df.to_csv('resumes_features.csv')
df.head(2)

,Ссылка,Название,Коммандировка,"Опыт работы, лет",О себе,Образование,Интересы,Навыки,Образование-1,Образование-2,Уровень образование,Кол-во образований,Работа 1,Работа 2,Где работал?,Кол-во работ
0,https://hh.ru/resume/1aa91e410000fd3cda0039ed1...,Analyst,"Moscow, willing to relocate, prepared for occa...",Work experience 7 years 4 months,"Responsible, communicable, quick study and det...",Higher education\n2015\nMOSCOW STATE UNIVERSIT...,"Specializations:\nSales manager, account manag...",Key skills\nAnalitical thinking\nEnglish\nOrga...,MOSCOW STATE UNIVERSITY OF MECHANICAL ENGINEER...,,Higher education,1,"NUTRICIA\nMoscow, nutricia.ru\nFood Products.....","Volkswagen Group Russia\nMoscow, www.volkswage...","[NUTRICIA\nMoscow, nutricia.ru\nFood Products....",4
1,https://hh.ru/resume/628596ac000657935b0039ed1...,BI аналитик,"Москва, не готова к переезду, готова к редким ...",Опыт работы 1 год 5 месяцев,В последние годы проходила обучение без возмож...,Высшее образование (Бакалавр)\n2022\nНациональ...,"Специализации:\nBI-аналитик, аналитик данных\n...",Ключевые навыки\nTableau\nPower BI\nSQL\nMS Ex...,"Национальный исследовательский университет ""Вы...",,Высшее образование (Бакалавр),1,"Ozon\nМладший аналитик\nСоздание, поддержка и ...",OZON\nСтажер группы BI аналитики и отчетности\...,"[Ozon\nМладший аналитик\nСоздание, поддержка и...",3


На этом все завершено. Перейдем в следующий ноутбук. 